Package and file imports

In [2]:
!pip install tf-models-official 
!pip install -U tensorflow
!pip install -U tensorflow-text


  Using cached tf_models_official-2.11.2-py2.py3-none-any.whl (2.3 MB)
  Using cached Cython-0.29.32-py2.py3-none-any.whl (986 kB)
  Using cached gin_config-0.5.0-py3-none-any.whl (61 kB)
  Using cached google_api_python_client-2.70.0-py2.py3-none-any.whl (10.7 MB)
  Using cached immutabledict-2.2.3-py3-none-any.whl (4.0 kB)
  Using cached kaggle-1.5.12.tar.gz (58 kB)
  Preparing metadata (setup.py) ... done
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl (98 kB)
  Using cached opencv_python_headless-4.6.0.66-cp37-abi3-macosx_11_0_arm64.whl (30.0 MB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached pycocotools-2.0.6.tar.gz (24 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached sa

In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import bz2
import csv
import re
import os
import gc
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import shutil

import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer


tf.get_logger().setLevel('ERROR')

from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evanmeeks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/evanmeeks/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ModuleNotFoundError: No module named 'tensorflow_hub'

In [ ]:
#Wrapping entire file import process in an if-else statement to speed up processing if the cleaned files have already
# been exported

if not os.path.exists('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/cleaned_test_text.csv') and not os.path.exists('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/cleaned_train_text.csv'):

  #Reading in BZ2 Files
  train_file = bz2.BZ2File('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/train.ft.txt.bz2')
  test_file = bz2.BZ2File('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/test.ft.txt.bz2')

  #Splitting files into their individual lines
  train_file_lines = train_file.readlines()
  test_file_lines = test_file.readlines()

  #Decoding the lines into parseable data
  train_file_lines = [x.decode('utf-8') for x in train_file_lines]
  test_file_lines = [x.decode('utf-8') for x in test_file_lines]

  #Function to convert the individual lines into array objects. Note: 
  #Label 0 = negative, Label 1 = positive review. 

  def load_extract(file):
    texts, labels = [], []
    for line in file: 
      labels.append(int(line[9])-1)
      texts.append(line[10:].strip())
    return np.array(labels), texts

  #Calling the function on the data
  train_labels, train_text = load_extract(train_file_lines)
  test_labels, test_text = load_extract(test_file_lines)

  #Taking only 10% of datasets because sheer size keeps crashing the notebook.
  sub_train_text = train_text[:360000]
  sub_train_labels = train_labels[:360000]
  sub_test_text = test_text[:360000]
  sub_test_labels = test_labels[:360000]

  #Quick check that everything above worked smoothly.
  print(sub_train_text[0])
  print(sub_train_labels[0], "\n")
  print(sub_test_text[0])
  print(sub_test_labels[0])


else: 

  pass

In [ ]:
#Function to clean the data of stop words, URLs/Hyperlinks, numbers, whitespace, etc. 
def clean_texts(texts):
    stwords = stopwords.words('english')
    temp_texts = []
    for i in range(len(texts)):
        text = re.sub('\d','0',texts[i])
        if 'www.' in text or 'http:' in text or 'https:' in text or '.com' in text: 
            text = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", " ", text)
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = text.split()
        text = [word for word in text if not word in stwords]
        text = ' '.join(text)
        temp_texts.append(text)
    return temp_texts

Cleaning the datasets

In [ ]:
#Custom CSV file open/reader
def open_csv(file):
  with open(file, mode='r') as csv_file:
    result = []
    for line in csv_file: 
      result.append(line)
    return result

In [ ]:
#Creating the train/test text sets if they haven't already been exported

if os.path.exists('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/cleaned_train_text.csv'):

  train_texts = open_csv('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/cleaned_train_text.csv')
  sub_train_labels = open_csv('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/sub_train_labels.csv')

else: 
  
    train_texts = clean_texts(sub_train_text)
    np.savetxt('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/cleaned_train_text.csv', train_texts, fmt ='%s')
    np.savetxt('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/sub_train_labels.csv', sub_train_labels, fmt ='%s')


if os.path.exists('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/cleaned_test_text.csv'):

  test_texts = open_csv('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/cleaned_test_text.csv')
  sub_train_labels = open_csv('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/sub_test_labels.csv')

else: 

    test_texts = clean_texts(sub_test_text)
    np.savetxt('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/cleaned_test_text.csv', test_texts, fmt ='%s')
    np.savetxt('/Users/evanmeeks/Documents/data_education/Springboard/Github Projects/Capstone 3/sub_test_labels.csv', sub_test_labels, fmt ='%s')


**Data Modeling using a Multinomial and Logitistic Regression**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect.fit(train_texts)

train_texts_vec = count_vect.transform(train_texts)

test_texts_vec = count_vect.transform(test_texts)


In [ ]:
nb = MultinomialNB()
nb.fit(train_texts_vec, sub_train_labels)

In [ ]:
y_pred = nb.predict(test_texts_vec)

In [ ]:
print('Accuracy:', accuracy_score(sub_test_labels, y_pred))

In [ ]:
lr_model = LogisticRegression(n_jobs=-1, max_iter=150)
lr_model.fit(train_texts_vec, sub_train_labels)

In [ ]:
pred_lr = lr_model.predict(test_texts_vec)
print('Accuracy:', accuracy_score(sub_test_labels, pred_lr))

In [ ]:
a = 1201
sample = test_texts[a]
print(sample)

sample_vec = count_vect.transform([sample])
pred = lr_model.predict(sample_vec)
print('\npredicted label:',pred[0])
print('actual label:', test_labels[a])

**Data Modeling using Tensorflow's BERT (Bidirectional Encoder Representations from Transformers)**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_texts, sub_train_labels, shuffle = True)

In [ ]:

#Reading csv file into TensorFlow dataset and printing first element for verification
train_ds = tf.data.TextLineDataset(["cleaned_train_text.csv"])
test_ds = tf.data.TextLineDataset(["cleaned_test_text.csv"])

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']




In [ ]:
x=get_sentence_embedding([train_texts])
print(x)

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(np.array(X_train), y_train, epochs=3)